In [ ]:
!pip install facenet-pytorch

In [ ]:
# Import library yang diperlukan
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
from facenet_pytorch import MTCNN
import torch
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path ke dataset di Google Drive
train_folder = '/content/drive/MyDrive/nist_2/train'
val_folder = '/content/drive/MyDrive/nist_2/val'
train_augment_folder = '/content/drive/MyDrive/nist_2/sample_train_augment'

# Path ke folder output di Google Drive
output_train_folder = '/content/drive/MyDrive/output_nist_2_save/train'
output_val_folder = '/content/drive/MyDrive/output_nist_2_save/val'
output_augment_folder = '/content/drive/MyDrive/output_nist_2_save/sample_train_augment'
output_2metode_folder = '/content/drive/MyDrive/path_to_output_nist_2_save/duametode'

# Inisialisasi model MTCNN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=True, device=device)

# Waktu eksekusi
start_time = time.time()

# Fungsi untuk meningkatkan kualitas gambar
def enhance_image(image):
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    lab = cv2.cvtColor(blurred, cv2.COLOR_BGR2Lab)
    l, a, b = cv2.split(lab)
    l = cv2.equalizeHist(l)
    enhanced_image = cv2.merge((l, a, b))
    return cv2.cvtColor(enhanced_image, cv2.COLOR_Lab2BGR)

# Fungsi untuk mendeteksi wajah menggunakan OpenCV
def detect_face(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
    return image, faces

# Fungsi untuk mendeteksi wajah menggunakan MTCNN
def detect_face_mtcnn(image):
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    boxes, _ = mtcnn.detect(rgb_image)
    faces = []
    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = [int(coord) for coord in box]
            faces.append([x1, y1, x2, y2])
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return image, faces

# Fungsi untuk menghapus latar belakang
def remove_background(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, threshold = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, contours, 255)
    foreground = cv2.bitwise_and(image, image, mask=mask)
    return foreground, mask

# Fungsi untuk mengganti latar belakang
def change_background(image, mask, bg_color=(0, 255, 0)):
    new_background = np.full(image.shape, bg_color, dtype=np.uint8)
    result = cv2.bitwise_and(new_background, new_background, mask=cv2.bitwise_not(mask))
    result = cv2.add(result, image)
    return result

# Fungsi untuk menyimpan gambar hasil
def save_image(image, output_path):
    if isinstance(image, np.ndarray):
        cv2.imwrite(output_path, image)
    else:
        print("Error: Gagal menyimpan gambar. `image` bukan array NumPy.")

# Fungsi untuk menampilkan gambar dengan Matplotlib
def display_images(original, enhanced, face_detected1, face_detected2, remove_bg, replace_bg):
    plt.figure(figsize=(18, 6))
    plt.subplot(1, 6, 1)
    plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    plt.title('Original Image', fontsize=10)
    plt.axis('off')

    plt.subplot(1, 6, 2)
    plt.imshow(cv2.cvtColor(enhanced, cv2.COLOR_BGR2RGB))
    plt.title('Enhanced Image', fontsize=10)
    plt.axis('off')

    plt.subplot(1, 6, 3)
    plt.imshow(cv2.cvtColor(face_detected1, cv2.COLOR_BGR2RGB))
    plt.title('OpenCV', fontsize=10)
    plt.axis('off')

    plt.subplot(1, 6, 4)
    plt.imshow(cv2.cvtColor(face_detected2, cv2.COLOR_BGR2RGB))
    plt.title('MTCNN', fontsize=10)
    plt.axis('off')

    plt.subplot(1, 6, 5)
    plt.imshow(cv2.cvtColor(remove_bg, cv2.COLOR_BGR2RGB))
    plt.title('Remove BG', fontsize=10)
    plt.axis('off')

    plt.subplot(1, 6, 6)
    plt.imshow(cv2.cvtColor(replace_bg, cv2.COLOR_BGR2RGB))
    plt.title('New Background', fontsize=10)
    plt.axis('off')

    plt.tight_layout()
    plt.show()  # Menampilkan gambar di Google Colab

# Fungsi untuk memproses dan menyimpan gambar
def process_and_save_images(folder_path, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    total_images = 0
    face_detected_count_opencv = 0
    face_detected_count_mtcnn = 0
    no = 1

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".ppm"):
                image_path = os.path.join(root, file)
                original_image = cv2.imread(image_path)
                total_images += 1

                enhanced_image = enhance_image(original_image)
                face_detected_image_opencv, faces1 = detect_face(original_image.copy())
                face_detected_image_mtcnn, faces2 = detect_face_mtcnn(original_image.copy())
                image_no_bg, mask = remove_background(enhanced_image.copy())
                result = change_background(enhanced_image.copy(), mask, bg_color=(0, 255, 0))

                face_output_path = os.path.join(output_folder, f"face_detected_{no}.png")
                display_images(original_image, enhanced_image, face_detected_image_opencv, face_detected_image_mtcnn, image_no_bg, result)
                save_image(face_detected_image_opencv, face_output_path)
                no += 1

                if len(faces1) > 0:
                    face_detected_count_opencv += 1
                if len(faces2):
                    face_detected_count_mtcnn += 1

    # Menghitung dan menampilkan akurasi
    if total_images > 0:
        accuracy_opencv = (face_detected_count_opencv / total_images) * 100
        accuracy_mtcnn = (face_detected_count_mtcnn / total_images) * 100
        print(f"Total Images Processed: {total_images}")
        print(f"Images with Face Detected OpenCV: {face_detected_count_opencv}")
        print(f"Face Detection Accuracy (OpenCV): {accuracy_opencv:.2f}%")
        print(f"Images with Face Detected MTCNN: {face_detected_count_mtcnn}")
        print(f"Face Detection Accuracy (MTCNN): {accuracy_mtcnn:.2f}%")
    else:
        print("No images found.")

# Memproses dan menampilkan gambar di masing-masing folder
process_and_save_images(val_folder, output_2metode_folder)

# Waktu eksekusi
end_time = time.time()
execution_time = end_time - start_time
print(f"Total waktu eksekusi: {execution_time:.2f} seconds.")
print("Semua gambar telah diproses dan ditampilkan.\n")
